# Initialization of Data
This part uses the library pandas, that allows us to use an easy way to call the data from the tables. 
The excel file is located on the same folder and is called Dream9.xlsx. 
The excel file has two sheets: the first one for training data and the second one for scoring data.
The are stored as Dream9_training and Dream9_scoring respectively.
If you need to, you also have a variable called Dream9 that contains both tables.

I clasified the Variables that appear in the tables with 3 main clasifications:

**Dependent**: Those variables that appear on the training Data but not on the scoring Data

**Categorical**: Those variables that have non-numerical values. All of them with the exception on cyto.cat represent binary decitios, like yes or no.

**Protein**: The variables that contain data on the protein concentration.

To invoke combinations of those variables you can use the magic of Python. I found that a practival way can be:

    [v for v in All if (v not in Protein and v in Dependent and v <> 'resp.simple')]

You can read that as: *"Oh!, mighty variable that is a variable who has appeared in all the variables, if you are not a Protein, you are Dependent and you are not called "resp. simple" please appear among us"*

In [1]:
#Open the data and read in pandas
import pandas
Dream9_training=pandas.read_excel('Dream9.xlsx',"trainingData")
Dream9_scoring=pandas.read_excel('Dream9.xlsx',"scoringData")
Dream9=pandas.concat([Dream9_training,Dream9_scoring])

#Division of types of Variables
All=list(Dream9_training.keys())
Sc=list(Dream9_scoring.keys())

#Dependent variables are present in the training set but not in the scoring set
Dependent=[]
for v in All:
    if v not in Sc:
        Dependent+=[v]
print '**Dependent**:',Dependent,'\n'

#Categorical variables have discrete values and can't be measured by euclidean distances
Categorical=['SEX', 'PRIOR.MAL', 'PRIOR.CHEMO', 'PRIOR.XRT', 'Infection', 'cyto.cat', 
             'ITD', 'D835', 'Ras.Stat', 'resp.simple', 'Relapse', 'vital.status']
print '**Categorical**:',Categorical,'\n'

#The last 231 variables are proteins
Protein=All[-231:]
print '**Not Protein**:',[v for v in All if v not in Protein],'\n'

**Dependent**: [u'resp.simple', u'Relapse', u'vital.status', u'Overall_Survival', u'Remission_Duration'] 

**Categorical**: ['SEX', 'PRIOR.MAL', 'PRIOR.CHEMO', 'PRIOR.XRT', 'Infection', 'cyto.cat', 'ITD', 'D835', 'Ras.Stat', 'resp.simple', 'Relapse', 'vital.status'] 

**Not Protein**: [u'SEX', u'Age.at.Dx', u'AHD', u'PRIOR.MAL', u'PRIOR.CHEMO', u'PRIOR.XRT', u'Infection', u'cyto.cat', u'ITD', u'D835', u'Ras.Stat', u'resp.simple', u'Relapse', u'vital.status', u'Overall_Survival', u'Remission_Duration', u'WBC', u'ABS.BLST', u'BM.BLAST', u'BM.MONOCYTES', u'BM.PROM', u'PB.BLAST', u'PB.MONO', u'PB.PROM', u'HGB', u'PLT', u'LDH', u'ALBUMIN', u'BILIRUBIN', u'CREATININE', u'FIBRINOGEN', u'CD13', u'CD33', u'CD34', u'CD7', u'CD10', u'CD20', u'HLA.DR', u'CD19'] 



You can try calling variables below.
Also, to call part of the table you can do Table[Columns][rows]. For example:

    Dream9[Dependent][:5]

You will invoke a table containind the dependent variables in columns and only the first 5 items as rows.

In [2]:
[variable for variable in All if (variable not in Protein and variable in Dependent and variable <> 'resp.simple')]

[u'Relapse', u'vital.status', u'Overall_Survival', u'Remission_Duration']

In [3]:
Dream9[Dependent][:5]

,resp.simple,Relapse,vital.status,Overall_Survival,Remission_Duration
train_id_001,CR,No,A,568.57,564.14
train_id_002,CR,Yes,D,185.86,123.86
train_id_003,RESISTANT,NaN,D,56.29,NaN
train_id_004,CR,Yes,D,98.14,63.43
train_id_005,CR,Yes,A,454.71,97.57


# Data preprocessing
A important step before the prediction is the pre-processing of the data. It allows to create new variables that are more related to the dependent variables. Also we can modify the expected variables to have new results.

The tables containing the new data are saved in Qtraining.csv and Qscoring.csv

## Making all the variables quantitative
To make all the variables quantitative we take all the values that are yes/no, pos/neg F/M and convert them to 1 and 0. In the case that there is no data or not conclusive data we use NaN. 

In the special case of cyto.cat that has multiple values we create a column for each unique value. Each column contains 1 or 0 depending depending if the data corresponds to that unique value or not.

## Preprocessing data for proteins
Sometimes a low concentration, as well as a high concentration of the protein can cause illness. If both a high value and a low value can have the same effect on the dependent variables, then there will not be a good correlation. So we are comparing the protein against the mean and squaring the result. (What i mistakenly called normalized)

Since the mean is expected to be 0 because the data of all the proteins are mean centered, squaring the result should be similar to the "Normalization"

Keeping in mind that there may be a correlation if the mean is distant from the data, we also have the possibility of creating new variables, where the correlation between the dependent variables and this squared difference is maximized. (Not implemented yet)

## Cutoff of Dependent data.
If a patient lives longer than 2 years or has a remission longer than 2 years is considered as a category. Some patients have much longer periods, that may be independent of any variable, because the patient is still alive. We don't want this data interfering with the prediction, so we may suppose that all the patients that live longer than 2 years and a half (130 weeks) live just 130 weeks. We can apply the same logic for Remission


In [4]:
import numpy

###############
#  Functions  #
###############

def alias(Series,aliases):
    #Changes the values on a Series with aliases as a dict that transform the old values in the new values
    new_Series=(Series=='This creates and new series')
    for key,data in zip(Series.keys(),Series):
        new_Series[key]=False
        for val in aliases:
            try:
                if numpy.isnan(data):
                    new_Series[key]=numpy.nan
            except:
                pass             
            if data==val:
                new_Series[key]=aliases[val]
                break
    return new_Series

def split(Series,All_Data):
    #For Series with multiple values, creates a table with a column for each unique value
    #The value is True for the correct column and False for all the other columns
    D=[]
    for value in All_Data[Series.name].unique():
        q=(Series==value)
        q.name='%s=%s'%(q.name,value)
        D+=[q]
    return pandas.concat(D,axis=1)


def difference_from_mean(Table,All_Data):
    #This function creates a new Table with the values equal to (value-mean)/std
    #Since most of the values are already centered around 0 it woul be better to just take the square?
    D=[]
    for i,var in enumerate(Table.keys()):
        m=All_Data[var].mean()
        std=All_Data[var].std()
        D+=[(Table[var]-m)**2/std]
        D[i].name='%s_Normalized'%var
    return pandas.concat(D,axis=1)

def squared(Table):
    #This function squares all the values on a table
    D=[]
    for i,var in enumerate(Table.keys()):
        D+=[Table[var]**2]
        D[i].name='%s_Squared'%var
    return pandas.concat(D,axis=1)

def cutoff(Series,cutoff):
    #This function makes values above a threeshold equal to the threeshold
    new_Series=Series.copy()
    for key,data in zip(Series.keys(),Series):
        if data>cutoff:
            new_Series[key]=cutoff
        else:
            new_Series[key]=data
    new_Series.name='%s_cut'%Series.name
    return new_Series
    
####################
#  Pre-processing  #
####################    
    
def PreProcess(table):
    #Select A
    Tables=[table[[v for v in table.keys() if v not in Categorical]]]
    
    #Convert yes/no to 1/0
    Alias_Dict={'SEX':{'F':1},'PRIOR.MAL':{'YES':1},'PRIOR.CHEMO':{'YES':1},'PRIOR.XRT':{'YES':1},
                'Infection':{'Yes':1},'ITD':{'POS':1,'ND':numpy.nan},'D835':{'POS':1,'ND':numpy.nan},
                'Ras.Stat':{'POS':1,'NotDone':numpy.nan},'resp.simple':{'CR':1},'Relapse':{'Yes':1},
                'vital.status':{'A':1}}
    Aliased=[]
    for key in Alias_Dict:
        if key in table.keys():
            Aliased+=[alias(table[key],Alias_Dict[key])]
    Tables+=[pandas.concat(Aliased,axis=1)]
    
    #Split data that has multiple values
    Tables+=[split(table['cyto.cat'],Dream9)]
    
    #Create new data for protein
    #Tables+=[difference_from_mean(table[Protein],Dream9)]
    
    #Create new data for protein
    Tables+=[squared(table[Protein])]
    
    #Join everything
    
    Cut=[]
    for key in ['Overall_Survival','Remission_Duration']:
        if key in table.keys():
            Cut+=[cutoff(table[key],130)]
    if len(Cut)>0:
        Tables+=[pandas.concat(Cut,axis=1)]

    return pandas.concat(Tables,axis=1)
    
#Create the new tables
Q_training=PreProcess(Dream9_training)
Q_scoring=PreProcess(Dream9_scoring)

#Save the tables as csv
Q_training.to_csv('Qtraining.csv')
Q_scoring.to_csv('Qscoring.csv')

#Number of columns and rows of new Table
print Q_training.shape
print Q_scoring.shape

(187, 516)
(70, 509)


# Correlations
It is useful to find the variables that have most correlation with other variables. If some of the new variables we created are important, they may be shown on this table.

In [5]:
#Correlation for Continuous variables
Corr=pandas.DataFrame()
Q_Dependent=Dependent+['Overall_Survival_cut','Remission_Duration_cut']
for Variable in Q_Dependent:
    C=Q_training[[t for t in Q_training.keys() if (t not in Q_Dependent)]+[Variable]].corr()[Variable][:-1]
    Corr=Corr.append(C)
#Most important variables for prediction
Variables_for_prediction={'pCR':'resp.simple','pRelapse':'Relapse','OS':'Overall_Survival_cut','Remission':'Remission_Duration_cut'}
Selected_variables={}
for Pred in Variables_for_prediction:
    Variable=Variables_for_prediction[Pred]
    print Corr.T.sort(Variable)[Variable][Corr.T.sort(Variable)[Variable]**2>0.2**2]
    Selected_variables.update({Pred:Corr.T.sort(Variable)[Variable][Corr.T.sort(Variable)[Variable]**2>0.2**2].index})

NPM1.3542          -0.252036
CD34               -0.241693
ERG                -0.215514
PIK3CA              0.208228
cyto.cat=diploid    0.216598
Name: resp.simple, dtype: float64
HGB                  -0.215277
D835                 -0.214264
SMAD5                -0.202269
EGFR.pY992_Squared    0.216344
GAPDH_Squared         0.223614
TP53                  0.232152
ITD                   0.243735
CD13                  0.265287
Name: Relapse, dtype: float64
HSP90AA1_B1        -0.278365
Age.at.Dx          -0.277940
HNRNPK             -0.238282
cyto.cat=-5,-7     -0.226312
EIF2S1             -0.213093
PA2G4.pS65         -0.201985
BECN1              -0.200825
ITGA2               0.205998
WTAP_Squared        0.209763
H3K27Me3            0.212390
FN1                 0.218321
HSPA1A_L            0.229288
DIABLO_Squared      0.235278
TRIM62              0.247285
cyto.cat=diploid    0.258477
MAPT                0.261157
YAP1p               0.263526
ALBUMIN             0.279360
HGB                 0

# Prediction
In order to predict I created a Prediction Class. The good thing about Classes is that you can create new classes that can heredate the propierties and methods of other Classes. That means that we have don't have to write this part of the code again. So what does this code does?

## Creates groups

The class contains a function to divide the 

    Prediction.split(n)
    
Splits the training set in n parts. n-1 parts will go to the training set, while 1 part will be on the testing set. We create n groups, so each of the n parts can be in the testing set. For example if we split the testing sets in 3 parts (n=3) We will be returned 3 groups that will contain:

|Testing|Training|
|-------|--------|
|1      |2,3     |
|2      |1,3     |
|3      |1,2     |

The output is a list of dictionaries. Here are some examples of how to call the groups:
    
    groups=Prediction.split(n) #Assign the group to a variable
        [{'test':<Table>,'train'<Table>},{'test':<Table>,'train'<Table>},...]
    g=groups[0] #Returns the frist dictionary
        {'test':<Table>,'train'<Table>}
    g['train'] #Returns the training set

    Prediction.create_groups(rep,[n])
    
Will create n groups rep times. This method will invoke split rep times creating a bigger list. It is useful to have more testing and training groups to have a more reliable scoring.
    
## Predicts

### Main prediction function
    Prediction.predict(scoring=Scoring_set,out='prediction.csv')
    
Will give results about the Scoring set using the full training set with the prediction methods used.

### Prediction methods

    Prediction.pCR(training,testing): 
Returns a Series containing the probability of Complete Remission ('resp.simple')
        
    Prediction.pRelapse(training,testing): 
Returns a Series containing the probability of Relapse ('Relapse')
        
    Prediction.Remission(training,testing): 
Returns a Series with the estimated Remission Duration ('Remission_Duration') in weeks (not binned)
        
    Prediction.OS(training,testing): 
Returns the estimated Overall Survival ('Overall_Survival') in weeks (not binned)

### Reduced prediction methods.
The prediction algorithms for pCR and pRelapse should be similar, as well as the algorithms gor Remission and Overall Survival. We can have only two prediction algorithms: pPred for the probabilities prediction and qPred for the time Prediction

    Prediction.pPred(training,testing,var):
Returns the Probability prediction as a Series. 
Should return a value between 0 and 1.
Used for pCR and pRelapse
        
    Prediction.qPred(self,training,testing,var):
Quantitative prediction, returns the number of weeks.
Used for Remission and OS
It will be binned after this, so if this is a binned prediction, multiply the result by 52
 
    Prediction._bin(data,bins=[0,52,104])
Bins the results as 0,1 2,3. The 0 values correspond to numpy.nan values.

## Scoring functions
    Prediction.accuracy(self,rep=1,groups=[]):
Returns the Balanced accuracy prediction for all the functions
    
    Prediction.result(self,groups=[],alpha=0.5,Measure='All',rep=1):    
Returns the scoring calculations for the prediction as a Table
You can select wich scoring to return with Measure
alpha allows to select the scoring that is lower than alpha*100% of the scores, supossing a normal distribution.
If alpha is 0.5 it will return the mean

    Prediction.score(self,groups=[],Measure='All'):
Main scoring function
Returns a dictionary for all the scores
You can select a Measure
Will create groups if no group given.

### Balanced accuracy (BAC)
The Balanced accuracy is an improved scoring function over the accuracy. While the accuracy measures the number of correct gueses over the entire number of guesses, the balanced accuracy measures the number of correct guesses in each subgroup of possible outcomes and then obtains a mean. For example in a population where P=99 and N=1, a random guess where we say that all the population is P would give 99% accuracy, while only having 50% balanced accuracy.

|Total population| Positive (P) | Negative (N) |
|--|--|--|
|Predicted positive| True positive (TP)| False positive (FP)| 
|Predicted necative| False negative(FN)| True negative (TN)|

#### Probabilities
For the Balanced accuracy we define a that the predicted positive values are all those values that have a probability greater or equal than 0.5, the rest being predicted negative.

$$BAC=\frac{TP}{P}+\frac{TN}{N}$$

#### Categorical
BAC is not strictly defined for continuous variables, so to estimate this scoring method we used the binned results. We have three variables, so a True negative is not defined. We define TP as correct guesses, where the test bin is the same as the predicted bin.

$$BAC=\frac{1}{n} \sum_{n} \frac{TP}{P}$$

### Area under the receiver operating characteristic curve (AUROC)
AUROC is a complimentary analysis to BAC. The ROC curve is a curve that compares the tradeoff between sensitivity and specificity. It create a cutoff by supossing a cutoff (k) on the prediction, where values greater or equal than k will be considered positive, and values smaller than k are considered negative. 

At k=1 all the predictions are negative, and there are no false negatives nor True positives (We are at point (0,0). As k decreases the number of True positives also increases. 

If there is a k value where all True positives have been found and there are no false positives, we will be at the point (1,0). If this point is reached AUROC will be 1. 

When k=0 all predictions are positive and we will arrive at point (1,1).

AUROC measures the area under this curve and is not defined for the Categorical dependent variables, since we don't have a parameter "k" to modify.
<img src=https://upload.wikimedia.org/wikipedia/commons/8/8c/Receiver_Operating_Characteristic.png>

### Pearson correlation coefficient (PCC)
Measures the correlation between the predicted values and the actual values. It originally return a value between -1 and 1, but is normalized to be between 0 and 1. For Remission duration and Overall Survival it computes with the real results (not binned). If your prediction return binned results you can unbin them by multiplying by 52 and adding 26.

$$PCC = \frac{\sum_{i=1}^{n}(p_i-\overline{p})(a_i-\overline{a})}{\sqrt{\sum_{i=1}^{n}(p_i-\overline{p})^2}\sqrt{\sum_{i=1}^{n}(a_i-\overline{a})^2}}$$

$$PCC_{norm} = (PCC + 1)/2$$

### Concordance Index (CI)
For each pair of results, it calculates if the predicted results are in the same order than the expected results. For example: for a pair (51,60),(64,62), where 51 and 64 are the predicted values, and 60 and 62 are the expected values. If $51<64$ and $60<62$ (The same relationsip), then $h_{ij}=0$.

$$\sum_{i<j}^{N}h(i,j)$$

The pairs that are counted are not censored pairs. A pair is censored if the Overall survival of i is lower than j, but i is alive. If j is equal to i and i or j is alive it is also censored. Nevertheless, if the Overall Survival of j is lower and i and j is alive it is not censored (...Please tell me if you can understand that). A similar relation is applied to Relapse Status with Remission, so it does not matter whether the patients were alive or dead.

In [6]:
import numpy as np
import random
import pandas
from scipy.stats import norm
Training_set=Q_training
Scoring_set=Q_scoring
Variables_for_prediction={'pCR':'resp.simple','pRelapse':'Relapse','OS':'Overall_Survival','Remission':'Remission_Duration'}
class Prediction:
    ########################
    ##   Initialization   ##
    ########################
    def __init__(self,training=Training_set,PredVar=Variables_for_prediction):
        self.training=training
        self.PredVar=PredVar
    
    ######################
    ##  Group Creation  ##
    ######################
    
    def create_groups(self,rep,n=5):
        #Creates rep*n training and testing groups
        Groups=[]
        for i in range(rep):
            Groups+=self.split(n)
        return Groups
    
    def split(self,n=5):
        #Divides the training groups n times, to have training and testing groups.
        #The testing group will contain 1/n of the data
        #It returns a list of dictionaries.
        #To call a group use for example groups[0]['test']
        #That will return the first testing group
        training_keys=self.training.T.keys().get_values().copy()
        random.shuffle(training_keys)
        sublist=np.array_split(training_keys,n)
        groups=[]
        for i in range(n):
            train=[]
            test=[]
            for j in range(n):
                sl=list(sublist[j])
                if j<>i:
                    train+=sl
                else:
                    test+=sl
            train.sort()
            test.sort()
            groups+=[{'train':self.training.loc[train],'test':self.training.loc[test]}]
        return groups
    
    
    ####################
    ##   Prediction   ##
    ####################
    
    def predict(self,scoring=Scoring_set,out='prediction.csv'):
        #Main prediction function.
        #Returns the result of the prediction for the Scoring set as a csv
        #Also prints the results
        with open(out,'w+') as handle:
            handle.write('#Patient_id, pCR, pRelapse, Remission, OS\n')
            print '#Patient_id, pCR, pRelapse, Remission, OS'
            Data=zip(scoring.T.keys(),
                     self.pCR(self.training,scoring),
                     self.pRelapse(self.training,scoring),
                     self._bin(self.Remission(self.training,scoring)),
                     self._bin(self.OS(self.training,scoring)))
            for d in Data:
                handle.write(','.join(str(k) for k in d)+'\n')
                print ','.join(str(k) for k in d)
    
    
    def pCR(self,training,testing): 
        #Returns the probability of Complete Remission ('resp.simple')
        return self.pPred(training,testing,'pCR')
        
    def pRelapse(self,training,testing): 
        #Returns the probability of Relapse ('Relapse')
        return self.pPred(training,testing,'pRelapse')
        
    def Remission(self,training,testing): 
        #Returns the estimated Remission Duration ('Remission_Duration') in weeks (not binned)
        return self.qPred(training,testing,'Remission')
        
    def OS(self,training,testing): 
        #Returns the estimated Overall Survival ('Overall_Survival') in weeks (not binned)
        return self.qPred(training,testing,'OS')
    
    def pPred(self,training,testing,var):
        #Probability prediction, should return a value between 0 and 1.
        #Used for pCR and pRelapse
        avg_p=sum(training[self.PredVar[var]] == True)/float(len(training))
        #print avg_p
        return pandas.Series([avg_p+random.random()*1E-6 for i in range(len(testing))],index=testing.index)
        
    def qPred(self,training,testing,var):
        #Quantitative prediction, returns the number of weeks.
        #Used for Remission and OS
        #It will be binned after this, so if this is a binned prediction, multiply the result by 52
        count=np.bincount(self._bin(training[self.PredVar[var]]))
        count[0]=0 #bin 0 is nan, do not count nan
        val=range(len(count))
        val.sort(key=lambda i: count[i],reverse=True)
        mode=(val[0]-1)*52.0+26
        return pandas.Series([mode+random.random()*1E-2 for i in range(len(testing))],index=testing.index)
    
    def _bin(self,data,bins=[0,52,104]):
        #This function will bin the results from Remission and Overall Survival as expected    
        bins = np.array(bins)
        digitized = numpy.digitize(data, bins)
        for i,v in enumerate(data):
            if np.isnan(v):
                digitized[i]=0
        return digitized

    
    ######################
    ## Scoring methods  ##
    ######################
    
    def accuracy(self,rep=1,groups=[]):
        #Returns the Balanced accuracy prediction for all the functions
        groups=self.create_groups(rep) if groups==[] else groups
        S=self.score(Measure='BAC',groups=groups)
        return {s:numpy.mean(S[s]['BAC']) for s in S}
    
    def result(self,rep=1,groups=[],alpha=0.5,Measure='All'):    
        #Returns the scoring calculations for the prediction
        #You can select wich scoring to return with Measure
        #alpha allows to select the scoring that is lower than alpha*100% of the scores, supossing a normal distribution.
        #If alpha is 0.5 it will return the mean
        groups=self.create_groups(rep) if groups==[] else groups
        S=self.score(groups,Measure)
        T=pandas.concat([pandas.Series({key:numpy.mean(S[v][key])-norm.ppf(alpha)*numpy.std(S[v][key]) for key in S[v]},name=v) for v in S],axis=1)
        return T.T
        #return  {'pCR':pCR_Error,'pRelapse':pRelapse_Error,'Remission':Remission_Error,'OS':OS_Error}
        #pRelapse_Error
    
    def score(self,groups=[],Measure='All'):
        #Main scoring function
        #Returns a dictionary for all the scores
        #You can select a Measure
        #Will create groups if no group given.

        #Define groups if not defined
        groups=self.split() if groups==[] else groups
        
        #Initialize score dictionary
        lg=range(len(groups))
        S={'pCR':{},'pRelapse':{},'Remission':{},'OS':{}}
        [S[v].update({'BAC':[0 for r in lg],'PCC':[0 for r in lg]})for v in S]
        [S[v].update({'AUROC':[0 for r in lg]})for v in ['pCR','pRelapse']]
        [S[v].update({'CI':[0 for r in lg]})for v in ['OS','Remission']]
        #Independent=[v for v in self.training.keys() if v not in Dependent]
        for i,g in enumerate(groups):
            train=g['train']
            test=g['test']#[Independent]
            
            #Train and evaluate
            pCR_values=self.pCR(train,test)
            pRelapse_values=self.pRelapse(train,test)
            Remission_values=self.Remission(train,test)
            OS_values=self.OS(train,test)
            
            #Compare the expeced values to the obtained values
            #test=g['test']
            
            #Calculate the BAC scores
            if Measure in ['All','BAC']:
                S['pCR']['BAC'][i]=self._pBAC(pCR_values,test['resp.simple'])
                S['pRelapse']['BAC'][i]=self._pBAC(pRelapse_values,test['Relapse'])
                S['Remission']['BAC'][i]=self._cBAC(self._bin(Remission_values),self._bin(test['Remission_Duration']))
                S['OS']['BAC'][i]=self._cBAC(self._bin(OS_values),self._bin(test['Overall_Survival']))
                if Measure=='BAC':
                    continue
            
            #Calculate the PCC scores
            if Measure in ['All','PCC']:
                S['pCR']['PCC'][i]=self._pPCC(pCR_values,test['resp.simple'])
                S['pRelapse']['PCC'][i]=self._pPCC(pRelapse_values,test['Relapse'])
                S['Remission']['PCC'][i]=self._pPCC(Remission_values,test['Remission_Duration'])
                S['OS']['PCC'][i]=self._pPCC(OS_values,test['Overall_Survival'])
                if Measure=='PCC':
                    continue
            
            #Calculate the AUROC scores
            if Measure in ['All','AUROC']:
                S['pCR']['AUROC'][i]=self._AUROC(pCR_values,test['resp.simple'])
                S['pRelapse']['AUROC'][i]=self._AUROC(pRelapse_values,test['Relapse'])
                if Measure=='AUROC':
                    continue
            
            #Calculate the CI scores
            if Measure in ['All','CI']:
                S['Remission']['CI'][i]=self._RCI(Remission_values,test)
                S['OS']['CI'][i]=self._OSCI(OS_values,test)
                if Measure=='CI':
                    continue           
        
        return S
    
    def _pBAC(self,predicted,expected):
        #print predicted
        #print expected
        TP=float(((predicted>=0.5) & (expected>=0.5)).dropna().sum())
        TN=float(((predicted<0.5) & (expected<0.5)).dropna().sum())
        P=max(float((expected>=0.5).dropna().sum()),1E-64)
        N=max(float((expected<0.5).dropna().sum()),1E-64)
        return (TP/P+TN/N)/2

    
    def _cBAC(self,predicted,expected):
        TV=[]
        a=set(expected)
        for val in set(a):
            if val>0:
                TP=float(((predicted==val) & (expected==val)).sum())
                P=max(float((expected==val).sum()),1E-64)
                TV+=[TP/P]
        return numpy.mean(TV)
        
    def _AUROC(self,predicted,expected):
        AUC=0
        TPRl=0
        FPRl=0
        K=list(set(numpy.concatenate((predicted,[1.0,0.0]))))
        K.sort(reverse=True)
        #print K
        for k in K:
            T=(predicted>=k)
            TP=((expected==1) & (T==1)).sum()
            FP=((expected==0) & (T==1)).sum()
            TPR=TP/max(float(expected.sum()),1E-64)#may be 0 sometimes
            FPR=FP/max(float((expected==0).sum()),1E-64)#may be 0 sometimes   
            AUC+=TPRl*(FPR-FPRl)
            FPRl=FPR
            TPRl=TPR
        return AUC
            
    
    def _OSCI(self,predicted,expected):        
        c=0.0
        H=0.0
        for i,ai,pi,Ai in zip(range(len(predicted)),predicted,expected['Overall_Survival'],expected['vital.status']):
            if numpy.isnan(ai) or numpy.isnan(pi):
                continue
            for j,aj,pj,Aj in zip(range(len(predicted)),predicted,expected['Overall_Survival'],expected['vital.status']):
                if i>=j:
                    continue
                if numpy.isnan(aj) or numpy.isnan(pj):
                    continue
                if ai<=aj and Ai: #The patient i has smaller Survival but is still alive
                    continue
                if aj==ai and Aj: #The patient j has smaller Survival but is still alive
                    continue
                
                if numpy.sign(round(ai-aj,5))==numpy.sign(round(pi-pj,5)):
                    #print ai,aj,pi,pj
                    H+=1
                    c+=1
                else:
                    #H+=0
                    c+=1
        #print H,c
        return H/c
                    
    def _RCI(self,predicted,expected):
        c=0.0
        H=0.0
        for i,ai,pi,Ai in zip(range(len(predicted)),predicted,expected['Remission_Duration'],expected['Relapse']):
            if numpy.isnan(ai) or numpy.isnan(pi):
                continue
            for j,aj,pj,Aj in zip(range(len(predicted)),predicted,expected['Remission_Duration'],expected['Relapse']):
                if i>=j:
                    continue
                if numpy.isnan(aj) or numpy.isnan(pj): #no value on Relapse
                    continue
                if ai<=aj and not Ai: #The patient i has smaller Remission but has not Relapsed
                    continue
                if aj==ai and not Aj: #The patient j has smaller Remission but has not Relapsed
                    continue
                
                if numpy.sign(round(ai-aj,5))==numpy.sign(round(pi-pj,5)):
                    #print ai,aj,pi,pj
                    H+=1
                    c+=1
                else:
                    #H+=0
                    c+=1
        #print H,c
        return H/c
    
    def _pPCC(self,predicted,expected): #Pearson Correlation Coeficient
        A=pandas.concat([predicted,expected],axis=1,keys=['pred','exp'])
        A=A.dropna() #drop data that has na as a result
        p=A['pred'].mean()
        a=A['exp'].mean()
        sp=max(((A['pred']-p)**2).sum()**0.5,1E-64) #Sometimes this is 0, then S=0 too
        sa=max(((A['exp']-a)**2).sum()**0.5,1E-64) #Sometimes this is 0, then S=0 too
        S=(A['pred']-p)*(A['exp']-a)
        #print S.sum(),sp,sa
        return (S.sum()/sp/sa+1)/2

Dummy=Prediction()
print Dummy.result(alpha=0.5,Measure='All',rep=5)
#Dummy.predict()
#Dummy._pPCC(pandas.Series([-200,+50,-200,+120,-120,+100,-200]),pandas.Series([0,2,0,1,1,3,0]))
#print Dummy.accuracy(100)
#print Dummy.accuracy(100)
#print Dummy.accuracy(100)

              AUROC       BAC        CI       PCC
pCR        0.526783  0.500000       NaN  0.518470
pRelapse   0.488708  0.500000       NaN  0.485310
OS              NaN  0.333333  0.546221  0.471703
Remission       NaN  0.340000  0.531452  0.484652

[4 rows x 4 columns]


# Linear Regression
To modify the previous class you can change the Qpred and Cpred methods.
For the linear regression I also modified the create_groups methods, since the Linear regression algorithm will not accept NaN values. You cn also modify the \__init\__ class to incorporate variables used at the initialization of the class.

In [7]:
from sklearn import datasets, linear_model

Good_variables={'pCR': [u'KDR_Squared', u'ATF3', u'RPS6_Squared', u'cyto.cat=Misc', u'GATA3', u'CDKN2A_Squared', u'NF2.pS518', u'CASP9.cl315_Squared', u'IGFBP2', u'SMAD3_Squared', u'PRKAA1_2.pT172_Squared', u'HDAC3_Squared', u'CLPP', u'PRIOR.MAL', u'ATG7', u'cyto.cat=diploid', u'DLX1_Squared', u'MSI2', u'CCNE2', u'NPM1.3542', u'ARC', u'cyto.cat=21', u'ITGAL', u'SMAD2_Squared', u'RPS6.pS240_244', u'MYC', u'LCK_Squared', u'ITGA2', u'GAPDH', u'CCNE1', u'PA2G4.pT70_Squared', u'cyto.cat=-7', u'MTOR.pS2448_Squared', u'CD44', u'PRKCB.II_Squared', u'MAP2K1_2.pS217_221_Squared', u'BAD.pS136_Squared', u'CASP9.cl330', u'GSKA_B.pS21_9', u'CTSG', u'FOXO3_Squared', u'TGM2', u'STAT3.pS727', u'CASP8_Squared', u'PIK3CA', u'RPS6', u'SFN', u'PTK2_Squared', u'ZNF296_Squared', u'PRKCD.pT507', u'Age.at.Dx', u'STMN1_Squared', u'YWHAZ_Squared', u'HSPB1', u'STMN1', u'PDK1.pS241_Squared', u'CDK1', u'MAPK9'],
                'pRelapse': [u'cyto.cat=t9;22', 'IGFBP2_Squared', u'CCND3', u'KIT_Squared', u'PTEN.pS380T382T383', u'BCL2_Squared', u'BAK1_Squared', u'SMAD5.pS463_Squared', 'MDM2', 'ARC', u'PTPN11_Squared', u'H3histon_Squared', u'PA2G4.pS65_Squared', 'HDAC1_Squared', u'EIF2S1.pS51._Squared'], 
                'OS': [u'PRIOR.MAL', u'ARC', u'cyto.cat=diploid', u'H3histon', u'Age.at.Dx', u'PTGS2_Squared', u'SMAD4', u'PA2G4.pS65', u'STMN1', u'EIF2AK2', u'H3K27Me3', u'HSP90AA1_B1'], 
                'Remission': [u'CASP9.cl330', u'ERG', u'ALBUMIN', u'CASP3.cl175', u'TP53', u'RPS6KB1.pT389', u'PLAC1', u'JMJD6', u'SMAD3_Squared', u'ERG_Squared', u'TRIM24', u'Age.at.Dx', u'HSPA1A_L', u'ATG7_Squared', u'ARC_Squared', u'STAT3.pS727', u'CBL_Squared', u'BIRC5_Squared', u'ARC', u'YWHAE', u'SMAD5.pS463', u'BRAF_Squared', u'MTOR.pS2448_Squared']}

All_variables={'pCR':[v for v in Q_training.keys() if v not in Q_Dependent],
               'pRelapse':[v for v in Q_training.keys() if v not in Q_Dependent],
               'Remission':[v for v in Q_training.keys() if v not in Q_Dependent],
               'OS':[v for v in Q_training.keys() if v not in Q_Dependent]}

Variables_for_prediction={'pCR':'resp.simple','pRelapse':'Relapse','OS':'Overall_Survival','Remission':'Remission_Duration'}

class LinearRegression(Prediction):
    def __init__(self,training=Training_set,pivot=All_variables,PredVar=Variables_for_prediction):
        self.training=training
        self.pivot=pivot
        self.ols=linear_model.LinearRegression()
        self.PredVar=PredVar
        
    def create_groups(self,rep):
        Groups=[]
        for i in range(rep):
            Groups+=self.split()
        New_Groups=[]
        for g in Groups:
            training=g['train']
            testing=g['test']
            Accept=True
            for dep in All_variables.keys():
                ind=All_variables[dep]
                #ind=pandas.concat([training[ind],testing[ind]]).dropna(axis=1).keys()
                A=pandas.concat([training[ind],pandas.DataFrame(training[self.PredVar[dep]])],axis=1,keys=['ind','dep'])
                A=A.dropna()
                if len(A['dep',])<=0:
                    Accept=False
            if Accept:
                New_Groups+=[g]
        return New_Groups
    
    def pPred(self,training,testing,dep):
        ind=self.pivot[dep] #Select independent variables
        #ind=pandas.concat([training[ind],testing[ind]]).dropna(axis=1).keys() #drop variables that have na
        A=pandas.concat([training[ind],pandas.DataFrame(training[self.PredVar[dep]])],axis=1,keys=['ind','dep'])
        A=A.dropna() #drop data that has na as a result
        self.ols.fit(A['ind',],A['dep',]) #train
        test=testing[ind].dropna()
        Results=self.ols.predict(test).T[0] #predict
        for i,val in enumerate(Results):
            if val>1:
                Results[i]=1
            if val<0:
                Results[i]=0
        R=pandas.Series(Results,index=test.index)
        return pandas.Series(R,index=testing.index)
        
    def qPred(self,training,testing,dep):
        ind=self.pivot[dep] #Select independent variables
        #ind=pandas.concat([training[ind],testing[ind]]).dropna(axis=1).keys() #drop variables that have na
        A=pandas.concat([training[ind],pandas.DataFrame(training[self.PredVar[dep]])],axis=1,keys=['ind','dep'])
        A=A.dropna() #drop data that has na as a result
        self.ols.fit(A['ind',],A['dep',]) #train
        test=testing[ind].dropna()
        Results=self.ols.predict(test).T[0] #predict
        for i,val in enumerate(Results):
            if val<0:
                Results[i]=0
        R=pandas.Series(Results,index=test.index)
        return pandas.Series(R,index=testing.index)
    
LR=LinearRegression(pivot=Good_variables)
print Dummy.result(rep=10)
print LR.result(rep=10)
Groups=LR.create_groups(10)

              AUROC       BAC        CI       PCC
pCR        0.485996  0.500000       NaN  0.489981
pRelapse   0.514349  0.500000       NaN  0.515842
OS              NaN  0.333333  0.556484  0.494843
Remission       NaN  0.333333  0.541715  0.497595

[4 rows x 4 columns]
              AUROC       BAC        CI       PCC
pCR        0.948839  0.867869       NaN  0.882124
pRelapse   0.779851  0.662110       NaN  0.725374
OS              NaN  0.469243  0.682276  0.724936
Remission       NaN  0.484968  0.675286  0.732501

[4 rows x 4 columns]


# Evolution

From this point we should see which combination of variables (or parameters, depending or your algorithm) will be better.
I though that for the variables a three phased alghoritm may be useful.
On the first phase we can check wether each variable can predict accurately anything.
On the second phase we can check if combination of variables can predict better than any single variable.
On the third phase we can create new combination of variables doing bolder changes, like mixing groups,

In [8]:
#Initialization
#Make your best guess here
Variables_for_prediction={'pCR':'resp.simple','pRelapse':'Relapse','OS':'Overall_Survival_cut','Remission':'Remission_Duration_cut'}

Good_variables={'pCR': [u'KDR_Squared', u'ATF3', u'RPS6_Squared', u'cyto.cat=Misc', u'GATA3', u'CDKN2A_Squared', u'NF2.pS518', u'CASP9.cl315_Squared', u'IGFBP2', u'SMAD3_Squared', u'PRKAA1_2.pT172_Squared', u'HDAC3_Squared', u'CLPP', u'PRIOR.MAL', u'ATG7', u'cyto.cat=diploid', u'DLX1_Squared', u'MSI2', u'CCNE2', u'NPM1.3542', u'ARC', u'cyto.cat=21', u'ITGAL', u'SMAD2_Squared', u'RPS6.pS240_244', u'MYC', u'LCK_Squared', u'ITGA2', u'GAPDH', u'CCNE1', u'PA2G4.pT70_Squared', u'cyto.cat=-7', u'MTOR.pS2448_Squared', u'CD44', u'PRKCB.II_Squared', u'MAP2K1_2.pS217_221_Squared', u'BAD.pS136_Squared', u'CASP9.cl330', u'GSKA_B.pS21_9', u'CTSG', u'FOXO3_Squared', u'TGM2', u'STAT3.pS727', u'CASP8_Squared', u'PIK3CA', u'RPS6', u'SFN', u'PTK2_Squared', u'ZNF296_Squared', u'PRKCD.pT507', u'Age.at.Dx', u'STMN1_Squared', u'YWHAZ_Squared', u'HSPB1', u'STMN1', u'PDK1.pS241_Squared', u'CDK1', u'MAPK9'],
                'pRelapse': [u'cyto.cat=t9;22', 'IGFBP2_Squared', u'CCND3', u'KIT_Squared', u'PTEN.pS380T382T383', u'BCL2_Squared', u'BAK1_Squared', u'SMAD5.pS463_Squared', 'MDM2', 'ARC', u'PTPN11_Squared', u'H3histon_Squared', u'PA2G4.pS65_Squared', 'HDAC1_Squared', u'EIF2S1.pS51._Squared', u'HGB', u'TP53', u'GAPDH_Squared', u'ERG', u'SIRT1_Squared', u'BCL2L1', u'XIAP', u'EGFR.pY992_Squared', u'HDAC3', u'ODC1_Squared'], 
                'OS': [u'SMAD2.pS465_Squared', u'EIF2S1', u'Age.at.Dx', u'NF2_Squared', u'DIABLO_Squared', u'cyto.cat=-5,-7', u'RPS6KB1', u'MAPT', u'HSPA1A_L', u'HGB', u'MAPK9_Squared', u'ERG_Squared', u'VHL_Squared', u'SMAD2.pS245_Squared', u'PARK7', u'CTSG_Squared', u'WTAP_Squared', u'TRIM62', u'ZNF296', u'AIFM1_Squared', u'DLX1', u'cyto.cat=diploid', u'H3K27Me3', u'H3K4Me2_Squared', u'BAD.pS155', u'CCND3', u'HSP90AA1_B1', u'YAP1p', u'GSKA_B.pS21_9_Squared', u'FN1', u'BAD.pS136', u'ALBUMIN', u'AKT1_2_3.pT308'], 
                'Remission': [u'CASP9.cl330', u'ERG', u'ALBUMIN', u'CASP3.cl175', u'TP53', u'RPS6KB1.pT389', u'PLAC1', u'JMJD6', u'SMAD3_Squared', u'ERG_Squared', u'TRIM24', u'Age.at.Dx', u'HSPA1A_L', u'ATG7_Squared', u'ARC_Squared', u'STAT3.pS727', u'CBL_Squared', u'BIRC5_Squared', u'ARC', u'YWHAE', u'SMAD5.pS463', u'BRAF_Squared', u'MTOR.pS2448_Squared']}

Best_LR=LinearRegression(pivot=Good_variables,PredVar=Variables_for_prediction)
groups=Best_LR.create_groups(50)
Best_result=Best_LR.accuracy(groups=groups)
Best_LR_result=Best_LR.score(groups=groups,Measure='BAC')
print Best_LR.result(rep=100)

              AUROC       BAC        CI       PCC
pCR        0.951366  0.859863       NaN  0.885330
pRelapse   0.796439  0.735013       NaN  0.753998
OS              NaN  0.522617  0.711447  0.709072
Remission       NaN  0.435919  0.688941  0.740880

[4 rows x 4 columns]


In [13]:
#Good_variables.update({'pCR': [u'KDR_Normalized', u'ATF3', u'RPS6_Normalized', u'cyto.cat=Misc', u'GATA3', u'CDKN2A_Normalized', u'NF2.pS518', u'CASP9.cl315_Normalized', u'IGFBP2', u'SMAD3_Normalized', u'PRKAA1_2.pT172_Normalized', u'HDAC3_Normalized', u'CLPP', u'PRIOR.MAL', u'ATG7', u'cyto.cat=diploid', u'DLX1_Normalized', u'MSI2', u'CCNE2', u'NPM1.3542', u'ARC', u'cyto.cat=21', u'ITGAL', u'SMAD2_Normalized', u'RPS6.pS240_244', u'MYC', u'LCK_Normalized', u'ITGA2', u'GAPDH', u'CCNE1', u'PA2G4.pT70_Normalized', u'cyto.cat=-7', u'MTOR.pS2448_Normalized', u'CD44', u'PRKCB.II_Normalized', u'MAP2K1_2.pS217_221_Normalized', u'BAD.pS136_Normalized', u'CASP9.cl330', u'GSKA_B.pS21_9', u'CTSG', u'FOXO3_Normalized', u'TGM2', u'STAT3.pS727', u'CASP8_Normalized', u'PIK3CA', u'RPS6', u'SFN', u'PTK2_Normalized', u'ZNF296_Normalized', u'PRKCD.pT507', u'Age.at.Dx', u'STMN1_Normalized', u'YWHAZ_Normalized', u'HSPB1', u'STMN1', u'PDK1.pS241_Normalized', u'CDK1', u'MAPK9']})
#Good_variables.update({'pRelapse': [u'cyto.cat=t9;22', 'IGFBP2_Normalized', u'CCND3', u'KIT_Normalized', u'PTEN.pS380T382T383', u'BCL2_Normalized', u'BAK1_Normalized', u'SMAD5.pS463_Normalized', 'MDM2', 'ARC', u'PTPN11_Normalized', u'H3histon_Normalized', u'PA2G4.pS65_Normalized', 'HDAC1_Normalized', u'EIF2S1.pS51._Normalized', u'SMAD6_Normalized', u'AKT1_Normalized']})
#Good_variables.update({'OS':['Age.at.Dx','HGB','cyto.cat=diploid','H3K27Me3','ARC']})
#Good_variables.update({'Remission':['ARC','SIRT1_Normalized','HGB','Age.at.Dx','ELK1.pS383_Normalized','cyto.cat=diploid','GAPDH_Normalized','STMN1','ERG','CCND1','CD13','HSPA1A_L','TP53']})

import time

i=0 #Iteration number counter
Stop=time.time()+60*60*1 #Stop time in seconds

Update_Best=False #Only update best if conditions apply

#Try to make better predictions
while time.time()<Stop:  
    i=i+1  
    #Change the variables
    Test_variables={}
    for key in Selected_variables:
        #Choose variable pool:
        Rnd=random.random()
        if Rnd>0.9:
            Pool=All_variables #Contains everything, they won't help mostly
        elif Rnd>0.3:
            #Something in between (but costs a bit to compute)
            cutoff=0.2-0.2*(Rnd-0.3)**.5/(0.9-0.3)**.5
            Pool={}
            for Pred in Variables_for_prediction:
                Variable=Variables_for_prediction[Pred]
                Pool.update({Pred:Corr.T.sort(Variable)[Variable][Corr.T.sort(Variable)[Variable]**2>cutoff**2].index})
        else:
            Pool=Selected_variables #Best variables  
        #Choose the change
        Rnd=random.random()
        if Rnd>0.7: #Sometimes drop a variable
            l=len(Good_variables[key])
            n=max(l-random.randint(1,5),1)
            Test_variables.update({key:random.sample(Good_variables[key],n)})
        elif Rnd>0.2: #Sometimes add a value
            varis=[v for v in Pool[key] if v not in Good_variables[key]]
            l=len(varis)
            n=min(random.randint(1,5),l)
            Test_variables.update({key:Good_variables[key]+random.sample(varis,n)})
        else: #Sometimes choose something completely random
            l=min(len(Pool[key]),len(Good_variables[key])*2)
            Test_variables.update({key:random.sample(Pool[key],random.randint(1,l))})

    #Create a linear regression
    LR=LinearRegression(pivot=Test_variables,PredVar=Variables_for_prediction)
    result=LR.accuracy(10)
    
    #Test against best prediction
    Next=True
    for key in result:
        if result[key]>Best_result[key]:
            Next=False
    if Next:
        continue
    
    #Test how many times it has a better result
    LR_result=LR.score(groups=groups,Measure='BAC')
    for key in LR_result.keys():
        conf=0.0
        new_BACs=LR_result[key]['BAC']
        old_BACs=Best_LR_result[key]['BAC']
        new_BACs.sort()
        old_BACs.sort()
        pos=0.0
        tot=float(len(new_BACs)**2)
        for new in new_BACs:
            for k,old in enumerate(old_BACs):
                if old>new:
                    pos+=k
                    break
        conf=pos/tot
        
        #If more than 60% are better, update
        if conf>0.6:
            with open('Good_variables.txt','a+') as handle:
                handle.write('Updated: '+key+'\n')
            Good_variables.update({key:Test_variables[key]})
            Update_Best=True
    
    #Update variables
    if Update_Best:
        Best_LR=LR
        groups=Best_LR.create_groups(50)
        Best_result=Best_LR.accuracy(groups=groups)
        Best_BAC_result=Best_LR.score(groups=groups,Measure='BAC')
        with open('Good_variables.txt','a+') as handle:
            handle.write(str(Good_variables)+'\n')
            handle.write(str(Best_result)+'\n')
        Update_Best=False 

In [ ]:
print i
print Best_LR.result(rep=100)

In [ ]:
Best_LR.predict()